In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Aug 18 14:12:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

In [3]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [ ]:
# fashion_mnist = tf.keras.datasets.fashion_mnist
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# X_train = X_train.astype(dtype) / 255.0
# y_train = y_train.astype(dtype)
# X_test = X_test.astype(dtype)  / 255.0
# y_test = y_test.astype(dtype)

# X_train = np.reshape(X_train, (-1, 784))
# X_test = np.reshape(X_test, (-1, 784))

In [14]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

170508288/170498071 [==============================] - 6s 0us/step


In [7]:
# from sklearn.model_selection import train_test_split

# glass = pd.read_csv('glass.data', header=None)
# y = glass.pop(10)
# X = glass
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# X_train = X_train.values.astype(dtype)
# X_test = X_test.values.astype(dtype)
# y_train = np.reshape(y_train.values, (-1, 1)).astype(dtype)
# y_test = np.reshape(y_test.values, (-1, 1)).astype(dtype)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [32]:
class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # The scaling vector could be [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        # and the resulting weighted values could be
        #
        # [[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_vector = tf.cumsum(tf.constant(self.regularization_penalty, shape=(x.shape[-1],), dtype=dtype), axis=0)
        weighted_values = scaling_vector * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class SSLayer(tf.keras.Model):
    def __init__(self, input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer, regularize=True):
        super().__init__()

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = SSRegularizer(self.regularization_penalty, self.regularization_method)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = SSLayer(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy


class SSModel(tf.keras.Model):
    def __init__(self, layer_sizes, activation=None, regularization_penalty=0.01, regularization_method='weighted_l1', kernel_initializer='glorot_uniform', bias_initializer='zeros'):
        super().__init__()
        
        self.sslayers = list()
        for l in range(len(layer_sizes) - 1):
            input_units = layer_sizes[l]
            units = layer_sizes[l + 1]
            if l < len(layer_sizes) - 2:
                layer = SSLayer(input_units, units, activation, regularization_penalty, regularization_method, kernel_initializer, bias_initializer)
            else:  # Last layer
                layer = SSLayer(input_units, units, 'softmax', 0., regularization_method, kernel_initializer, bias_initializer)
            self.sslayers.append(layer)

    def call(self, inputs):
        x = inputs
        for layer in self.sslayers:
            x = layer(x)
        return x
    
    def get_layer_sizes(self):
        layer_sizes = list()
        for l in range(len(self.sslayers)):
            layer = self.sslayers[l]
            layer_sizes.append(layer.W.shape[0])
            if l == len(self.sslayers) - 1:  # Last layer
                layer_sizes.append(layer.W.shape[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def print_neurons(self):
        for layer in self.sslayers[:-1]:
            print(get_param_string(layer.W, layer.b))
    
    def remove_regularization(self):
        for l in range(len(self.sslayers)):
            self.sslayers[l] = self.sslayers[l].copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.sslayers[0].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for l in range(0, len(self.sslayers) - 1):  # Every layer except of the last is regularized
            self.sslayers[l].regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, percentage, min_new_neurons, scaling_factor=0.001):
        for l in range(len(self.sslayers) - 1):
            layer1 = self.sslayers[l]
            layer2 = self.sslayers[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * percentage))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] #* scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)

In [38]:
def get_param_string(weights, bias):
    param_string = ""
    weights_with_bias = tf.concat([weights, tf.reshape(bias, (1, -1))], axis=0)
    max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
    magnitudes = np.floor(np.log10(max_parameters))
    for m in magnitudes:
        if m > 0:
            m = 0
        param_string += str(int(-m))
    return param_string


def print_epoch_statistics(model, x, y, validation_data, print_neurons):
    x_val = validation_data[0]
    y_val = validation_data[1]

    y_pred = model(x)
    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y, y_pred))
    accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y, y_pred))
    
    y_val_pred = model(x_val)
    val_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_val, y_val_pred))
    val_accuracy = tf.reduce_mean(tf.keras.metrics.sparse_categorical_accuracy(y_val, y_val_pred))
    print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
    hidden_layer_sizes = model.get_hidden_layer_sizes()
    print(f"hidden layer sizes: {hidden_layer_sizes}, total neurons: {sum(hidden_layer_sizes)}")
    if print_neurons:
        model.print_neurons()

    return val_loss


def train_model(model, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, 
                pruning_threshold=0.001, regularization_penalty_multiplier=1., growth_percentage=0.2, print_neurons=False):
    train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

    best_val_loss = np.inf
    training_stalled = False
    for epoch in range(epochs):
        print("##########################################################")
        print(f"Epoch {epoch + 1}/{epochs}")

        if epoch < self_scaling_epochs:
            print("Before growing:")

            val_loss = print_epoch_statistics(model, x, y, validation_data, print_neurons)
            if val_loss >= best_val_loss:
                if not training_stalled:
                    penalty = model.get_regularization_penalty() * regularization_penalty_multiplier
                    model.set_regularization_penalty(penalty)
                    training_stalled = True
            else:
                best_val_loss = val_loss
                training_stalled = False

            model.grow(percentage=growth_percentage, min_new_neurons=min_new_neurons, scaling_factor=pruning_threshold)
            print("After growing:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
        
        if epoch == self_scaling_epochs:
            model.remove_regularization()

        for step, (x_batch, y_batch) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                y_pred = model(x_batch, training=True)
                loss_value = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
                loss_value += sum(model.losses)

            grads = tape.gradient(loss_value, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        if epoch < self_scaling_epochs:
            print("Before pruning:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
            model.prune(threshold=pruning_threshold)
            print("After pruning:")
            print_epoch_statistics(model, x, y, validation_data, print_neurons)
        else:
            print_epoch_statistics(model, x, y, validation_data, print_neurons)

# Efficient training

## Cifar 10 dataset

In [18]:
epochs = 10
self_scaling_epochs = 10
batch_size = 32
min_new_neurons = 20

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.7915027141571045 - accuracy: 0.11311999708414078 - val_loss: 2.766880750656128 - val_accuracy: 0.11949999630451202
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/10
Before growing:
loss: 2.771711587905884 - accuracy: 0.09074000269174576 - val_loss: 2.757340908050537 - val_accuracy: 0.09019999951124191 - penalty: 0.0001
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.7717113494873047 - accuracy: 0.09074000269174576 - val_loss: 2.757340669631958 - val_accuracy: 0.09019999951124191 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.831411361694336 - accuracy: 0.3278200030326843 - val_loss: 1.832931637763977 - val_accuracy: 0.32679998874664307 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.8334648609161377 - accuracy: 0.32638001441955566 - val_loss: 1.834941029548645 - val_accuracy: 0.32429999113082886 - penalty: 0.0001
layer sizes: [3072, 14, 59, 38, 60, 10]
##########################################################
Epoch 2/10
Before growing:
loss: 1.8334648609161377 - accuracy: 0.3263800144

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.00001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.701932430267334 - accuracy: 0.09616000205278397 - val_loss: 2.6937167644500732 - val_accuracy: 0.10040000081062317
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
2222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.00001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/10
Before growing:
loss: 2.739136219024658 - accuracy: 0.0854400023818016 - val_loss: 2.7438509464263916 - val_accuracy: 0.08449999988079071 - penalty: 1e-05
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.739136219024658 - accuracy: 0.0854400023818016 - val_loss: 2.7438507080078125 - val_accuracy: 0.08449999988079071 - penalty: 1e-05
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.5972710847854614 - accuracy: 0.42989999055862427 - val_loss: 1.6050333976745605 - val_accuracy: 0.4284000098705292 - penalty: 1e-05
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.5968269109725952 - accuracy: 0.4307200014591217 - val_loss: 1.6047106981277466 - val_accuracy: 0.4296000003814697 - penalty: 1e-05
layer sizes: [3072, 108, 299, 298, 299, 10]
##########################################################
Epoch 2/10
Before growing:
loss: 1.5968269109725952 - accuracy: 0.4307200014

In [19]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.00001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/10
Before growing:
loss: 2.6930086612701416 - accuracy: 0.10617999732494354 - val_loss: 2.6887967586517334 - val_accuracy: 0.10769999772310257 - penalty: 1e-05
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 2.6930086612701416 - accuracy: 0.10617999732494354 - val_loss: 2.6887967586517334 - val_accuracy: 0.10769999772310257 - penalty: 1e-05
hidden layer sizes: [360, 360, 360, 360], total neurons: 1440
Before pruning:
loss: 1.5993088483810425 - accuracy: 0.42566001415252686 - val_loss: 1.6107486486434937 - val_accuracy: 0.424699991941452 - penalty: 1e-05
hidden layer sizes: [360, 360, 360, 360], total neurons: 1440
After pruning:
loss: 1.5944675207138062 - accuracy: 0.42941999435424805 - val_loss: 1.6057299375534058 - val_accuracy: 0.4277999997138977 - penalty: 1e-05
hidden layer sizes: [99, 300, 300, 300], total neurons: 999
##########################################################
Ep

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 1000, 1000, 1000, 1000, 10], activation='selu', regularization_penalty=0.000001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

Epoch 1/10
Before growing:
loss: 2.662282705307007 - accuracy: 0.1032399982213974 - val_loss: 2.657341718673706 - val_accuracy: 0.10700000077486038
layer sizes: [3072, 1000, 1000, 1000, 1000, 10]
222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.000001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/10
Before growing:
loss: 2.653571605682373 - accuracy: 0.10862000286579132 - val_loss: 2.6458799839019775 - val_accuracy: 0.11469999700784683 - penalty: 1e-06
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.653571605682373 - accuracy: 0.10862000286579132 - val_loss: 2.6458799839019775 - val_accuracy: 0.11469999700784683 - penalty: 1e-06
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.44719660282135 - accuracy: 0.4853399991989136 - val_loss: 1.5112738609313965 - val_accuracy: 0.4641999900341034 - penalty: 1e-06
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.4471937417984009 - accuracy: 0.4853000044822693 - val_loss: 1.5112707614898682 - val_accuracy: 0.4641999900341034 - penalty: 1e-06
layer sizes: [3072, 300, 300, 300, 341, 10]
##########################################################
Epoch 2/10
Before growing:
loss: 1.4471937417984009 - accuracy: 0.48530000448

In [12]:
epochs = 50
self_scaling_epochs = 50
batch_size = 32
min_new_neurons = 50

In [16]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1)

##########################################################
Epoch 1/50
Before growing:
loss: 2.707078695297241 - accuracy: 0.10412000119686127 - val_loss: 2.7202649116516113 - val_accuracy: 0.10530000180006027 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 2.707078695297241 - accuracy: 0.10412000119686127 - val_loss: 2.7202649116516113 - val_accuracy: 0.10530000180006027 - penalty: 0.0001
hidden layer sizes: [360, 360, 360, 360], total neurons: 1440
Before pruning:
loss: 1.840686559677124 - accuracy: 0.33399999141693115 - val_loss: 1.84286367893219 - val_accuracy: 0.33480000495910645 - penalty: 0.0001
hidden layer sizes: [360, 360, 360, 360], total neurons: 1440
After pruning:
loss: 1.8411165475845337 - accuracy: 0.33410000801086426 - val_loss: 1.8433187007904053 - val_accuracy: 0.33309999108314514 - penalty: 0.0001
hidden layer sizes: [15, 58, 34, 62], total neurons: 169
##########################################################
Epo

KeyboardInterrupt: ignored

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1)

##########################################################
Epoch 1/50
Before growing:
loss: 2.9734561443328857 - accuracy: 0.0878399983048439 - val_loss: 2.957174301147461 - val_accuracy: 0.0892999991774559 - penalty: 0.0001
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.973456621170044 - accuracy: 0.0878399983048439 - val_loss: 2.957174301147461 - val_accuracy: 0.0892999991774559 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.8268446922302246 - accuracy: 0.3210600018501282 - val_loss: 1.8327789306640625 - val_accuracy: 0.3165000081062317 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.8263518810272217 - accuracy: 0.32137998938560486 - val_loss: 1.8324464559555054 - val_accuracy: 0.31679999828338623 - penalty: 0.0001
layer sizes: [3072, 15, 58, 34, 70, 10]
##########################################################
Epoch 2/50
Before growing:
loss: 1.8263518810272217 - accuracy: 0.321379989385

In [ ]:
%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1)

##########################################################
Epoch 1/50
Before growing:
loss: 2.72518253326416 - accuracy: 0.10886000096797943 - val_loss: 2.734980821609497 - val_accuracy: 0.10670000314712524 - penalty: 0.0001
layer sizes: [3072, 300, 300, 300, 300, 10]
After growing:
loss: 2.72518253326416 - accuracy: 0.10886000096797943 - val_loss: 2.734980821609497 - val_accuracy: 0.10670000314712524 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
Before pruning:
loss: 1.8462209701538086 - accuracy: 0.3119400143623352 - val_loss: 1.8498504161834717 - val_accuracy: 0.30640000104904175 - penalty: 0.0001
layer sizes: [3072, 360, 360, 360, 360, 10]
After pruning:
loss: 1.8453584909439087 - accuracy: 0.3140600025653839 - val_loss: 1.848866581916809 - val_accuracy: 0.30880001187324524 - penalty: 0.0001
layer sizes: [3072, 13, 62, 33, 64, 10]
##########################################################
Epoch 2/50
Before growing:
loss: 1.8453584909439087 - accuracy: 0.314060002565

In [26]:
epochs = 50
self_scaling_epochs = 50
batch_size = 32
min_new_neurons = 200

In [29]:
# SCALING FACTOR *= 100

%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1, growth_percentage=0.5)

##########################################################
Epoch 1/50
Before growing:
loss: 2.5309395790100098 - accuracy: 0.13176000118255615 - val_loss: 2.52573561668396 - val_accuracy: 0.1274999976158142 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 2.5339834690093994 - accuracy: 0.1315000057220459 - val_loss: 2.528728723526001 - val_accuracy: 0.12700000405311584 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
Before pruning:
loss: 1.8360446691513062 - accuracy: 0.3235200047492981 - val_loss: 1.8377223014831543 - val_accuracy: 0.3240000009536743 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
After pruning:
loss: 1.8354604244232178 - accuracy: 0.323419988155365 - val_loss: 1.8373503684997559 - val_accuracy: 0.3246999979019165 - penalty: 0.0001
hidden layer sizes: [14, 53, 37, 67], total neurons: 171
##########################################################
Epoch 2/

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f344e38ac20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Before pruning:
loss: 1.2014533281326294 - accuracy: 0.572380006313324 - val_loss: 1.3904670476913452 - val_accuracy: 0.5128999948501587 - penalty: 1.0000000000000002e-07
hidden layer sizes: [1777, 2694, 1965, 688], total neurons: 7124
After pruning:
loss: 1.1997673511505127 - accuracy: 0.5733399987220764 - val_loss: 1.3894532918930054 - val_accuracy: 0.5133000016212463 - penalty: 1.0000000000000002e-07
hidden layer sizes: [1397, 2434, 1834, 641], total neurons: 6306
##########################################################
Epoch 36/50
Before growing:
loss: 1.1997673511505127 - accuracy: 0.5733399987220764 - val_loss: 1.3894532918930054 - val_accuracy: 0.5133000016212463 - penalty: 1.0000000000000002e-07
hidden layer sizes: [1397, 2434, 1834, 641], total neurons: 6306
After growing:
loss: 1.1999669075012207 - accuracy: 0.5731599926948547 - val_loss: 1.3896238803863525 - val_accuracy: 0.5135999917984009 - penalty: 1.0000000000000002e-07
hidden layer sizes: [2095, 3651, 2751, 961], tota

KeyboardInterrupt: ignored

In [31]:
# SCALING FACTOR *= 1000

%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1, growth_percentage=0.5)

##########################################################
Epoch 1/50
Before growing:
loss: 2.5994713306427 - accuracy: 0.10595999658107758 - val_loss: 2.5933938026428223 - val_accuracy: 0.10350000113248825 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 3.0187888145446777 - accuracy: 0.1050800010561943 - val_loss: 3.009610652923584 - val_accuracy: 0.10440000146627426 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
Before pruning:
loss: 1.8526203632354736 - accuracy: 0.3181599974632263 - val_loss: 1.8550060987472534 - val_accuracy: 0.3163999915122986 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
After pruning:
loss: 1.8521884679794312 - accuracy: 0.31863999366760254 - val_loss: 1.8549309968948364 - val_accuracy: 0.31700000166893005 - penalty: 0.0001
hidden layer sizes: [16, 60, 35, 70], total neurons: 181
##########################################################
Epoch

KeyboardInterrupt: ignored

In [33]:
# SCALING FACTOR *= 1000, SCALED ON SINGLE AXIS

%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1, growth_percentage=0.5)

##########################################################
Epoch 1/50
Before growing:
loss: 2.597078561782837 - accuracy: 0.1146399974822998 - val_loss: 2.581150770187378 - val_accuracy: 0.11819999665021896 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 3.046143054962158 - accuracy: 0.10559999942779541 - val_loss: 3.03760027885437 - val_accuracy: 0.10509999841451645 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
Before pruning:
loss: 1.852358102798462 - accuracy: 0.31672000885009766 - val_loss: 1.853013277053833 - val_accuracy: 0.314300000667572 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
After pruning:
loss: 1.8524550199508667 - accuracy: 0.3180199861526489 - val_loss: 1.8531001806259155 - val_accuracy: 0.31679999828338623 - penalty: 0.0001
hidden layer sizes: [18, 58, 36, 73], total neurons: 185
##########################################################
Epoch 2/5

KeyboardInterrupt: ignored

In [37]:
# SCALED ON SINGLE AXIS

%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1, growth_percentage=0.5)

##########################################################
Epoch 1/50
Before growing:
loss: 2.5947258472442627 - accuracy: 0.11299999803304672 - val_loss: 2.617108106613159 - val_accuracy: 0.10949999839067459 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 2.5946528911590576 - accuracy: 0.11292000114917755 - val_loss: 2.6170480251312256 - val_accuracy: 0.1096000000834465 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000


KeyboardInterrupt: ignored

In [35]:
# SCALING FACTOR *= 100, SCALED ON SINGLE AXIS

%%time

model = SSModel(layer_sizes=[3072, 300, 300, 300, 300, 10], activation='selu', regularization_penalty=0.0001, regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data=(X_test_norm, y_test), regularization_penalty_multiplier=0.1, growth_percentage=0.5)

##########################################################
Epoch 1/50
Before growing:
loss: 2.6810338497161865 - accuracy: 0.09408000111579895 - val_loss: 2.670012950897217 - val_accuracy: 0.09539999812841415 - penalty: 0.0001
hidden layer sizes: [300, 300, 300, 300], total neurons: 1200
After growing:
loss: 2.709690570831299 - accuracy: 0.09126000106334686 - val_loss: 2.6982622146606445 - val_accuracy: 0.09449999779462814 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
Before pruning:
loss: 1.8405990600585938 - accuracy: 0.33528000116348267 - val_loss: 1.843217134475708 - val_accuracy: 0.3328000009059906 - penalty: 0.0001
hidden layer sizes: [500, 500, 500, 500], total neurons: 2000
After pruning:
loss: 1.8411425352096558 - accuracy: 0.3349199891090393 - val_loss: 1.8437937498092651 - val_accuracy: 0.33180001378059387 - penalty: 0.0001
hidden layer sizes: [13, 61, 37, 69], total neurons: 180
##########################################################
Epo

KeyboardInterrupt: ignored

## Glass dataset

**TODO test training while increasing the size of the model**

In [ ]:
epochs = 60
self_scaling_epochs = 40
batch_size = 32
min_new_neurons = 50

In [ ]:
%%time

model = SSModel(layer_sizes=[10, 100, 100, 100, 100, 8], activation='selu', regularization_penalty=0.001, 
                regularization_method='weighted_l1', kernel_initializer='lecun_normal')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)

train_model(model, X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
            min_new_neurons, validation_data=(X_test_norm, y_test), pruning_threshold=0.01, print_neurons=True)

##########################################################
Epoch 1/60
Before growing:
loss: 2.4186086654663086 - accuracy: 0.12751677632331848 - val_loss: 2.2934510707855225 - val_accuracy: 0.13846154510974884 - penalty: 0.001
hidden layer sizes: [100, 100, 100, 100], total neurons: 400
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
After growing:
loss: 2.6235435009002686 - accuracy: 0.20134228467941284 - val_loss: 2.5807840824127197 - val_accuracy: 0.26153847575187683 - penalty: 0.001
hidden layer sizes: [150, 150, 150, 150], total neurons: 600
1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

ResourceExhaustedError: ignored

## Keras

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(66, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(13, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(27, activation='selu', kernel_initializer='lecun_normal'),
    tf.keras.layers.Dense(8, activation='softmax', kernel_initializer='lecun_normal'),
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), metrics=['accuracy'])

In [ ]:
%%time

model.fit(X_train_norm, y_train, epochs=60, validation_data=(X_test_norm, y_test))

Epoch 1/60
5/5 [==============================] - 1s 35ms/step - loss: 2.2734 - accuracy: 0.1342 - val_loss: 1.5871 - val_accuracy: 0.4462
Epoch 2/60
5/5 [==============================] - 0s 6ms/step - loss: 1.5645 - accuracy: 0.5235 - val_loss: 1.4996 - val_accuracy: 0.5846
Epoch 3/60
5/5 [==============================] - 0s 7ms/step - loss: 1.3287 - accuracy: 0.6644 - val_loss: 1.2981 - val_accuracy: 0.6154
Epoch 4/60
5/5 [==============================] - 0s 6ms/step - loss: 1.0804 - accuracy: 0.7248 - val_loss: 1.0719 - val_accuracy: 0.6615
Epoch 5/60
5/5 [==============================] - 0s 6ms/step - loss: 0.8965 - accuracy: 0.7584 - val_loss: 0.9357 - val_accuracy: 0.6923
Epoch 6/60
5/5 [==============================] - 0s 10ms/step - loss: 0.7831 - accuracy: 0.7651 - val_loss: 0.8577 - val_accuracy: 0.7385
Epoch 7/60
5/5 [==============================] - 0s 8ms/step - loss: 0.7072 - accuracy: 0.7852 - val_loss: 0.7962 - val_accuracy: 0.7538
Epoch 8/60
5/5 [================